# Lab 1: Error Metrics with PMF on Distance & Similarity Scores

This notebook solves the assignment in Lab1_Errors_Metrics.pdf by:
- Loading four CSVs: distance_scores1, distance_scores2, similarity_scores1, similarity_scores2
- Computing error metrics (AE, RE, MAE, MSE, RMSE, MAPE) after mapping labels to numeric targets
- Visualizing PMF (normalized histograms) of error distributions
- Summarizing performance across datasets

In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set(style="whitegrid")

In [2]:
# Load and Explore the CSV Data
paths = {
    'distance_scores1': 'lab-1/distance_scores1.csv',
    'distance_scores2': 'lab-1/distance_scores2.csv',
    'similarity_scores1': 'lab-1/similarity_scores1.csv',
    'similarity_scores2': 'lab-1/similarity_scores2.csv',
}

def load_dataset(path):
    df = pd.read_csv(path)
    return df

frames = {name: load_dataset(path) for name, path in paths.items()}

summary = {}
for name, df in frames.items():
    summary[name] = {
        'shape': df.shape,
        'labels_counts': df['label'].value_counts().to_dict(),
        'head': df.head(3)
    }

summary

FileNotFoundError: [Errno 2] No such file or directory: 'lab-1/distance_scores1.csv'

In [ ]:
# Calculate Absolute Error (AE) and Relative Error (RE)
# We map labels to binary target: genuine -> 1, impostor/forged -> 0
# For similarity scores: predicted = score (higher -> genuine)
# For distance scores: predicted = 1 - score (lower distance -> higher predicted genuineness)

label_map = {'genuine': 1}

def to_binary_label(series):
    return series.map(lambda x: label_map.get(x, 0)).astype(float)

# Avoid division by zero in RE/MAPE by using epsilon where actual==0
EPS = 1e-12

def compute_errors(df, kind):
    y = to_binary_label(df['label'])
    if kind.startswith('similarity'):
        y_pred = df['score'].astype(float)
    else:
        y_pred = 1.0 - df['score'].astype(float)
    ae = np.abs(y_pred - y)
    re = ae / np.maximum(y, EPS)
    pe = ae / np.maximum(y, EPS) * 100.0
    return y, y_pred, ae, re, pe

errors = {}
for name, df in frames.items():
    y, y_pred, ae, re, pe = compute_errors(df, name)
    errors[name] = {
        'y': y,
        'y_pred': y_pred,
        'ae': ae,
        're': re,
        'pe': pe
    }

{key: (np.mean(val['ae']), np.mean(val['re']), np.mean(val['pe'])) for key, val in errors.items()}

In [ ]:
# Compute MAE, MSE, RMSE per dataset
metrics_mae_mse_rmse = {}
for name, e in errors.items():
    ae = e['ae']
    mae = float(np.mean(ae))
    mse = float(np.mean(ae**2))
    rmse = float(np.sqrt(mse))
    metrics_mae_mse_rmse[name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse}

metrics_mae_mse_rmse

In [ ]:
# Compute MAPE per dataset (exclude samples with actual==0 to avoid division by zero)
metrics_mape = {}
for name, e in errors.items():
    y = e['y']
    pe = e['pe']
    mask = y > 0
    mape = float(np.mean(pe[mask])) if np.any(mask) else np.nan
    metrics_mape[name] = {'MAPE': mape}

metrics_mape

In [ ]:
# Visualize Error Metrics and PMF of Errors
mae_vals = {k: v['MAE'] for k, v in metrics_mae_mse_rmse.items()}
mse_vals = {k: v['MSE'] for k, v in metrics_mae_mse_rmse.items()}
rmse_vals = {k: v['RMSE'] for k, v in metrics_mae_mse_rmse.items()}
mape_vals = {k: v['MAPE'] for k, v in metrics_mape.items()}

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

axes[0].bar(mae_vals.keys(), mae_vals.values())
axes[0].set_title('MAE by dataset')
axes[0].set_xticklabels(mae_vals.keys(), rotation=45, ha='right')

axes[1].bar(mse_vals.keys(), mse_vals.values())
axes[1].set_title('MSE by dataset')
axes[1].set_xticklabels(mse_vals.keys(), rotation=45, ha='right')

axes[2].bar(rmse_vals.keys(), rmse_vals.values())
axes[2].set_title('RMSE by dataset')
axes[2].set_xticklabels(rmse_vals.keys(), rotation=45, ha='right')

axes[3].bar(mape_vals.keys(), mape_vals.values())
axes[3].set_title('MAPE by dataset (genuine only)')
axes[3].set_xticklabels(mape_vals.keys(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

# PMF of errors using normalized histograms for each dataset
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()
for ax, (name, e) in zip(axes, errors.items()):
    ae = e['ae']
    counts, bins = np.histogram(ae, bins=50, range=(0,1))
    pmf = counts / counts.sum()
    bin_centers = 0.5 * (bins[1:] + bins[:-1])
    ax.bar(bin_centers, pmf, width=(bins[1]-bins[0])*0.9)
    ax.set_title(f'PMF of AE: {name}')
    ax.set_xlabel('Absolute Error')
    ax.set_ylabel('Probability')
plt.tight_layout()
plt.show()

In [ ]:
# Performance Comparison Across Datasets
rows = []
for name in errors.keys():
    mae = metrics_mae_mse_rmse[name]['MAE']
    mse = metrics_mae_mse_rmse[name]['MSE']
    rmse = metrics_mae_mse_rmse[name]['RMSE']
    mape = metrics_mape[name]['MAPE']
    rows.append({'dataset': name, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE_genuine_only': mape})

comparison_df = pd.DataFrame(rows).sort_values('MAE')
comparison_df